In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, Activation, MaxPool2D
from tensorflow.keras.layers import BatchNormalization, Flatten, Reshape, Conv2DTranspose, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow_datasets as tfds
import PIL


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)


Data set

In [4]:
nyu_data, nyu_data_info= tfds.load('nyu_depth_v2', shuffle_files=True,
                 download=False, with_info=True)

train_data_set, validation_data_set = nyu_data['train'], nyu_data['validation']


In [6]:
BUFFER_SIZE = 10
BATCH_SIZE = 8
NUM_EPOCHS = 5

train_data = train_data_set.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
validation_data = train_data_set.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

Hiperparametros

In [70]:
height = 480
width = 640
channels = 3

kernel = (3, 3)

latent_dim = 512

IA Definition

In [ ]:
inputs = Input(shape=(height, width, channels), name="inputs")
x = inputs

x = Conv2D(32, kernel, padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x = MaxPool2D((2, 2))(x)

x = Conv2D(64, kernel, padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x = MaxPool2D((2, 2))(x)

x = Conv2D(128, kernel, padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x = MaxPool2D((2, 2))(x)

x = Conv2D(256, kernel, padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x = MaxPool2D((2, 2))(x)

x = Flatten()(x)
units = x.shape[1]
x = Dense(latent_dim, name="latent")(x)
x = Dense(units)(x)
x = LeakyReLU(alpha=0.2)(x)
x = Reshape((30, 40, 256))(x)

x = Conv2DTranspose(256, kernel, strides=2, padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2DTranspose(128, kernel, strides=2, padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2DTranspose(64, kernel, strides=2, padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2DTranspose(1, kernel, strides=2, padding="same")(x)
x = BatchNormalization()(x)
x = Activation("sigmoid", name="outputs")(x)

outputs = x

autoencoder = Model(inputs, outputs)
autoencoder.compile(optimizer=Adam(1e-3), loss='binary_crossentropy')
autoencoder.summary()


In [ ]:
autoencoder.fit(
    train_data,
    epochs=NUM_EPOCHS,
    validation_data=validation_data
)

In [ ]:
autoencoder.save("convAutoencoder.h5")

In [2]:
loaded_model = tf.keras.models.load_model("convAutoencoder.h5")

# loaded_model.predict()


In [48]:
ds = train_data_set.shuffle(1).batch(1).take(1)

for example in ds:  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
  depth = example["depth"]
  image = example["image"]
  # print('img', image.shape, 'dep', depth.shape)

img = image


In [53]:
# result = ds.map(lambda image, label: image)

prediction = loaded_model.predict(img)

print('prediction', prediction.shape)

prediction_temp = np.empty((1,480,640))

for i in range(480):
    for j in range(640):
        for element in prediction[0][i][j]:
            np.append(prediction_temp[0][i][j], element)

prediction = prediction_temp

print('prediction', prediction.shape)


prediction (1, 480, 640, 1)
prediction (1, 480, 640)


In [50]:
print('depth', depth.shape)

depth (1, 480, 640)


In [58]:


plt.imsave("filename.jpg", depth[0], cmap='Greys')
